In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -R drive/MyDrive/shared/GPT/gpt gpt

In [ ]:
main_path = "drive/MyDrive/AI/GPT/"
repo_dir = "eclipse_netbeans/"
project_name = "eclipse"
tests_path = main_path + "tests/" + repo_dir
model = "gpt2"
pred_path = main_path + "predictions/" + repo_dir + "gpt2/"
result_path = main_path + "results/" + repo_dir

In [ ]:
import json

def readTests(projectName, foldId):
    oneArgTests = []
    with open(f"{tests_path}{projectName}_ArgRecTests_fold{foldId}.txt") as f:
        lines = f.read().split('\n')
        for line in lines[:-1]:
            oneArgTests.append(json.loads(line))
        lines = None
    return oneArgTests

In [ ]:
def toSingleArgRecTest(this):
    test = {}
    test['filePath'] = this['filePath']
    test['numArg'] = 1 if this['argPos'] != 0 else 0
    test['lex_context'] = this['lex_context']
    test['excode_context'] = this['excode_context']
    test['next_excode'] = [this['next_excode']]
    test['next_lex'] = [this['next_lex']]
    test['expected_excode'] = this['expected_excode']
    test['expected_lex'] = this['expected_lex']
    test['ignored'] = this['ignored']
    test['argRecTestList'] = [this]
    test['id'] = this['test_id']
    test['methodInvocClassQualifiedName'] = this['methodInvocClassQualifiedName']
    return test

In [ ]:
def allTestsToSingleArgRecTest(oneArgTests):
    tests = []
    for i in range(len(oneArgTests)):
        test = oneArgTests[i]
        # SKIP METHOD INVOCATIONS WITH NO ARGUMENT PASSED
        if test['argPos'] > 0:
            test = toSingleArgRecTest(test)
            tests.append(test)
    return tests

In [ ]:
import os

def readPredictions(projectName, foldId):
    predictions = []
    if os.path.isfile(f"{pred_path}{projectName}_ArgRecs_fold{foldId}.txt"):
        filePath = f"{pred_path}{projectName}_ArgRecs_fold{foldId}.txt"

    with open(filePath) as f:
        lines = f.read().split('\n')
        for line in lines[:-1]:
            predictions.append(json.loads(line))
        lines = None
    return predictions

In [ ]:
from gpt import preprocessor

def preprocess(target):
    target = preprocessor.empty_string_literal(target)
    target = preprocessor.remove_array_access_index(target)
    return target

In [ ]:
def matchesArg(expectedLex, result):
    if result == expectedLex:
        return True

    if '->' in expectedLex and '->' in result:
        return True

    if '->' in expectedLex and result == "<LAMBDA>":
        return True

    if '.this' in expectedLex:
        if matchesArg(expectedLex[expectedLex.index('.this')+1:], result):
            return True

    if '.this' in result:
        if matchesArg(expectedLex, result[result.index('.this')+1:]):
            return True

    if expectedLex.startswith('this.'):
        if matchesArg(expectedLex[5:], result):
            return True

    if result.startswith('this.'):
        if matchesArg(expectedLex, result[5:]):
            return True

    return False

In [ ]:
def canAcceptResult(test, result):
    test = test['argRecTestList'][0]

    expectedLex = test['expected_lex']

    expectedLex = preprocess(expectedLex)
    if '{' in expectedLex:
        expectedLex = expectedLex[:expectedLex.index('{')].rstrip()

    result = preprocess(result)
    if '{' in result:
        result = result[:result.index('{')].rstrip()
    if result.find('(') > 0:
        result = preprocessor.normalize_method_invocation(result)
    
    if matchesArg(expectedLex, result):
        return True

    alternateLex = None
    if 'methodAccessLex' in test:
        alternateLex = test['methodAccessLex']
    if 'objectCreationLex' in test:
        alternateLex = test['objectCreationLex']
    if alternateLex is not None and matchesArg(alternateLex, result):
        return True

    if 'staticMemberAccessLex' in test:
        if matchesArg(test['staticMemberAccessLex'], result):
            return True
            
    return False

In [ ]:
expressionTypes = ['NAME', 'METHOD_INVOC', 'FIELD_ACCESS', 'ARRAY_ACCESS', 'CAST', 'STRING_LIT', 'NUM_LIT', 'CHAR_LIT', 'TYPE_LIT', 'BOOL_LIT',
    'NULL_LIT', 'OBJ_CREATION', 'ARR_CREATION', 'THIS', 'SUPER', 'COMPOUND', 'LAMBDA', 'METHOD_REF']
expressionTypeDict = {}

for i in range(len(expressionTypes)):
    expressionTypeDict[expressionTypes[i]] = i

tops = [1, 3, 5, 10]

In [ ]:
import os
import shutil
from collections import defaultdict

os.makedirs('results/' + project_name, exist_ok=True)
os.makedirs('logs/' + project_name, exist_ok=True)

In [ ]:
dataFrame = defaultdict(list)

def updateTopKResult(test, results, k, adequateGeneratedCandidate, doPrintIncorrectPrediction, projectName):
    isOverallCorrectTopK = False
    for i in range(min(k, len(results))):
        if canAcceptResult(test, results[i]):
            isOverallCorrectTopK = True
            break

    argType = test['argRecTestList'][0]['argType'] if 'argType' in test['argRecTestList'][0] else 'null'
    if isOverallCorrectTopK:
        dataFrame[f'GPTActualTop{k}'].append(1)
        dataFrame[f'GPTActualTop{k}{argType}'].append(1)

        if not test['ignored']:
            dataFrame[f'GPTOverallTop{k}'].append(1)
            dataFrame[f'GPTOverallTop{k}{argType}'].append(1)

        if adequateGeneratedCandidate:
            dataFrame[f'GPTTop{k}'].append(1)
            dataFrame[f'GPTTop{k}{argType}'].append(1)
    else:
        dataFrame[f'GPTActualTop{k}'].append(0)
        dataFrame[f'GPTActualTop{k}{argType}'].append(0)

        if not test['ignored']:
            dataFrame[f'GPTOverallTop{k}'].append(0)
            dataFrame[f'GPTOverallTop{k}{argType}'].append(0)
            if doPrintIncorrectPrediction:
                outputFileName = f"{projectName}_incorrect_ArgRecTests_top_{k}.txt"
                with open(f'logs/{repo_dir}{outputFileName}', "a") as f:
                    f.write(json.dumps(test['argRecTestList'][0]) + '\n')
                    f.write('Predictions: ' + str(results) + '\n')

        if adequateGeneratedCandidate:
            dataFrame[f'GPTTop{k}'].append(0)
            dataFrame[f'GPTTop{k}{argType}'].append(0)

In [ ]:
from tqdm.notebook import tqdm as tqdm

COMPOUND_CONSIDERED = False
TEST_APIS = [
            #  "org.eclipse.swt",
            #  "java.awt",
            #  "javax.swing",
]
if len(TEST_APIS) > 0:
    model += "__lib"
TEST_LOCAL_ARG = None
if TEST_LOCAL_ARG is not None:
    if TEST_LOCAL_ARG:
        model += "__local_arg"
    else:
        model += "__not_local_arg"

rank_list = []

fold_list = []
for testFile in os.listdir(pred_path):
    if testFile.startswith(f"{project_name}_ArgRecs_fold"):
        fold_id = testFile[testFile.find("fold")+4:][0]
        fold_list.append(fold_id)

for fold_id in tqdm(fold_list):
    tests = allTestsToSingleArgRecTest(readTests(project_name, fold_id))
    predictions = readPredictions(project_name, fold_id)
    for i in range(min(len(tests), len(predictions))):
        test = tests[i]
        prediction_detail = predictions[i]
        dataFrame['Tested'].append(1)

        if test['numArg'] == 0:
            continue
        if len(TEST_APIS) > 0:
            is_target = False
            for target_api in TEST_APIS:
                if test['methodInvocClassQualifiedName'].startswith(target_api + '.'):
                    is_target = True
            if not is_target:
                continue
        if TEST_LOCAL_ARG is not None:
            if not predictions[i]['sufficient_candidates']:
                continue
            is_local_arg = False
            for j in range(len(test['next_lex'][0])):
                for k in range(len(test['next_lex'][0][j])):
                    candidate = test['next_lex'][0][j][k]
                    if candidate == test['expected_lex']:
                        candidate_locality = test['argRecTestList'][0]['candidates_locality'][j][k]
                        if candidate_locality >= 4:
                            is_local_arg = True
                            break
                if is_local_arg:
                    break
            if TEST_LOCAL_ARG != is_local_arg:
                continue

        dataFrame['Predicted'].append(1)
        if not test['ignored']:
            dataFrame['Predicted supported'].append(1)
        
        gptResults = prediction_detail['predictions']
        if not COMPOUND_CONSIDERED:
            if '<COMPOUND>' in gptResults:
                gptResults.remove('<COMPOUND>')

        rank = -1
        for k in range(min(10, len(gptResults))):
            if canAcceptResult(test, gptResults[k]):
                rank = k
        rank_list.append(rank)

        for k in tops:
            updateTopKResult(test, gptResults, k, prediction_detail['sufficient_candidates'], False, project_name)

        dataFrame["GPT's runtime"].append(prediction_detail['runtime'])
        argType = test['argRecTestList'][0]['argType'] if 'argType' in test['argRecTestList'][0] else None
        if argType is not None:
            dataFrame["ArgType"].append(expressionTypeDict[argType])

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
MRR = 0
for rank in rank_list:
    if rank < 0:
        MRR += 0
    else:
        MRR += 1/(rank + 1)
MRR /= len(rank_list)
print(MRR)

0.4322325334651104


In [ ]:
# import pickle

# with open('logs/dataframe.pkl', 'wb') as f:
#     pickle.dump(dataFrame, f)

In [ ]:
import numpy as np
import csv

def printTestResult():
    with open(f'results/{project_name}/arg_rec_{model}_log.txt', 'w') as f:
        f.write(f"Ran {len(dataFrame['Tested'])} tests successfully.\n")
        f.write(f"Predicted {len(dataFrame['Predicted'])} tests.\n")
        f.write(f"Predicted {len(dataFrame['Predicted supported'])} tests that were supported.\n")
        f.write(f"Skipped {len(dataFrame['Tested']) - len(dataFrame['Predicted'])} tests. They were not taken into account during evaluation.\n")
        gptRuntime = np.mean(dataFrame["GPT's runtime"])
        f.write(f"GPT's runtime: {gptRuntime}s\n")
        f.write(f"MRR: {MRR}\n")

    accuracyPerNumArg = []
    row = []
    row.append("Number of params")
    row.append("Percentage of distribution")
    for k in tops:
        row.append(f"GPT's top-{k} accuracy")
    for k in tops:
        row.append(f"Top-{k} precision")
    for k in tops:
        row.append(f"Top-{k} recall")
    accuracyPerNumArg.append(row)

    unique, counts = np.unique(dataFrame['ArgType'], return_counts=True)
    counts = counts / counts.sum()
    argTypeDict = defaultdict(float)
    for i in range(len(unique)):
        argTypeDict[unique[i]] = counts[i]

    for i in range(len(expressionTypes)):
        argType = expressionTypes[i]
        row = []
        row.append(argType)
        row.append(argTypeDict[i] * 100)
        for k in tops:
            row.append(np.mean(dataFrame[f"GPTTop{k}{argType}"]))
        for k in tops:
            row.append(np.mean(dataFrame[f"GPTOverallTop{k}{argType}"]))
        for k in tops:
            row.append(np.mean(dataFrame[f"GPTActualTop{k}{argType}"]))
        accuracyPerNumArg.append(row)

    row = []
    row.append("all")
    row.append("100")
    for k in tops:
        row.append(np.mean(dataFrame[f"GPTTop{k}"]))
    for k in tops:
        row.append(np.mean(dataFrame[f"GPTOverallTop{k}"]))
    for k in tops:
        row.append(np.mean(dataFrame[f"GPTActualTop{k}"]))
    accuracyPerNumArg.append(row)

    with open(f'results/{project_name}/arg_rec_{model}.csv', 'w') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in accuracyPerNumArg:
            csv_writer.writerow(row)

    with open(f'results/{project_name}/arg_rec_{model}_log.txt', 'a') as f:
        for k in tops:
            correctTestsCount = np.sum(dataFrame[f"GPTActualTop{k}"])
            f.write(f"Target showed up in top {k} recommendations in {correctTestsCount} tests.\n")

printTestResult()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
for text_file in os.listdir('results/' + project_name):
    shutil.copyfile('results/' + project_name + '/' + text_file, main_path + 'results/' + project_name + '/' + text_file)